### Código para conjuntar datos de PROFECO y obtener precios PACIC


In [10]:
#Librerías
import pandas as pd
import requests
import os
import patoolib
import datetime
import locale
import re
import plotly.graph_objects as go
import plotly.io as pio

# Locale para Mex
locale.setlocale(locale.LC_ALL, 'es_MX')

'es_MX'

In [11]:
#Cambiar directorio de trabajo
try:
    os.chdir("C:/Users/claudio.pacheco/Documents/")
except:
    os.chdir("C:/Users/claud/Documents/")
#Crear carpeta. Si ya existe, no la crea
os.makedirs("profeco1", exist_ok=True)

#Cambiar directorio de trabajo
os.chdir("C:/Users/claudio.pacheco/Documents/")

In [12]:
#Lista de urls
urls=["https://datos.profeco.gob.mx/datos_abiertos/file.php?t=976288eab7d5db41e8f99258b2f93a6b"
      ]

In [13]:
# # Define the destination folder for the downloaded files
# folder_path = "profeco1"

# for url in urls:
#     # Define the file name and path
#     file_name = "temp.rar"
#     file_path = os.path.join(folder_path, file_name)
    
#     try:
        # # Download the file
        # print(f"Descargando {url}...")
        # response = requests.get(url)
        
        # # Check if the response status is OK (200)
        # response.raise_for_status()

        # # Save the file to the destination folder
        # with open(file_path, 'wb') as f:
        #     f.write(response.content)
        
    #     print(f"Descarga completada: {url}")
        
    #     # Check if "temp.rar" exists before extracting
    #     if os.path.exists(file_path):
    #         # Extract the files from the RAR archive
    #         print(f"Extrayendo archivos...")
    #         patoolib.extract_archive(file_path, outdir=folder_path)
    #         print("Extracción completada")
    #         # Remove the RAR file
    #         os.remove(file_path)
    #     else:
    #         print(f"Archivo {file_path} no encontrado, omitiendo la extracción.")
        
    # except requests.RequestException as e:
    #     print(f"Error descargando {url}. Error: {e}")
    # except Exception as e:
    #     print(f"Error procesando {url}. Error: {e}")




In [14]:
#Concatenar archivos de las carpetas en un solo dataframe
#Lista de carpetas

carpetas = os.listdir("profeco1")
archivos = [os.listdir("profeco1/"+carpeta) for carpeta in carpetas]

In [15]:
archivos

[['01-2023_01.csv',
  '01-2023_02.csv',
  '02-2023_01.csv',
  '02-2023_02.csv',
  '03-2023_01.csv',
  '03-2023_02.csv',
  '04-2023_01.csv',
  '04-2023_02.csv',
  '05-2023_01.csv',
  '05-2023_02.csv',
  '06-2023_01.csv',
  '06-2023_02.csv',
  '07-2023_01.csv',
  '07-2023_02.csv',
  '08-2023_01.csv',
  '08-2023_02.csv',
  '09-2023_01.csv']]

In [16]:
#Lista de dfs
dfs = []

for i, folder in enumerate(carpetas):
    for j, file in enumerate(archivos[i]):
        # Path
        file_path = "profeco1/" + folder + "/" + file
        
        # Leer csvs
        temp_df = pd.read_csv(file_path, encoding="utf-8", names=[
            "producto", "presentacion", "marca", "categoria", "catalogo", "precio",
            "fecharegistro", "cadenacomercial", "giro", "nombrecomercial", "direccion",
            "estado", "municipio", "latitud", "longitud"
        ])
        
        # Pegar en lista
        dfs.append(temp_df)
        print(f"Concatenando archivo {j+1} de {len(archivos[i])}")

# concatenar en df
df = pd.concat(dfs, ignore_index=True)
#Eliminar acentos
df["producto"] = df["producto"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# Fltro por catálogo
print("Filtrando datos: Dejar solo productos PACIC")
pacic = df[df["catalogo"] == "PACIC"]
#Todo en minúsculas
print("Proceso terminado")

Concatenando archivo 1 de 17
Concatenando archivo 2 de 17
Concatenando archivo 3 de 17
Concatenando archivo 4 de 17
Concatenando archivo 5 de 17
Concatenando archivo 6 de 17
Concatenando archivo 7 de 17
Concatenando archivo 8 de 17
Concatenando archivo 9 de 17
Concatenando archivo 10 de 17
Concatenando archivo 11 de 17
Concatenando archivo 12 de 17
Concatenando archivo 13 de 17
Concatenando archivo 14 de 17
Concatenando archivo 15 de 17
Concatenando archivo 16 de 17
Concatenando archivo 17 de 17
Filtrando datos: Dejar solo productos PACIC
Proceso terminado


In [17]:
#Se crean funciones para el análisis de datos
def examinar(df):
    #Examinar productos únicos
    print("Productos únicos en la tabla: ")
    print(df["producto"].unique())
    print("Número de productos únicos: ", len(df["producto"].unique()))
    print("Número de registros: ", format(len(df), ",d"))

def filtro_pacic(df):
    filtro = ["ACEITE", "ARROZ", "ATUN", "AZUCAR", "CARNE RES","CEBOLLA", "CHILE FRESCO",
              "CARNE CERDO", "FRIJOL","HUEVO", "JABON DE TOCADOR", "JITOMATE", "LECHE ULTRAPASTEURIZADA",
                "LIMON", "MANZANA", "NARANJA", "PAN DE CAJA", "PAPA", "PAPEL HIGIENICO", "PASTA PARA SOPA",
                "CARNE POLLO", "SARDINA", "TORTILLA DE MAIZ", "ZANAHORIA"]
    return df[df["producto"].isin(filtro)]
    
def filtro_supermercado(df):
    filtro = ['MEGA SORIANA', 'WAL-MART','BODEGA AURRERA', 'CHEDRAUI',
              'HIPERMERCADO SORIANA', 'SUPERMERCADOS LEY','SORIANA SUPER', 'CASA LEY', 'SORIANA EXPRESS',
              'MERCADO SORIANA','LA COMER', 'SUMESA', 'WAL-MART EXPRESS', 'BODEGA AURRERA EXPRESS',
              'CHEDRAUI SELECTO', 'SORIANA', 'SUPER CHEDRAUI', 'CITY MARKET', 'FRESKO LA COMER','LEY (AUTOSERVICIO)',
              'LEY EXPRESS']

    # Filtro
    df_filtered = df[(df['giro'] == 'SUPERMERCADO / TIENDA DE AUTOSERVICIO') & (df['cadenacomercial'].isin(filtro))]

    # Crear "grupo" basado en "cadenacomercial"
    def categorize_chain(chain):
        if chain in ["WAL-MART", "WAL-MART EXPRESS", "BODEGA AURRERA", "BODEGA AURRERA EXPRESS"]:
            return "WALMART"
        elif chain in ["SORIANA SUPER", "MEGA SORIANA", "HIPERMERCADO SORIANA", "MERCADO SORIANA", "SORIANA EXPRESS", "SORIANA", "LA COMER", "SUMESA", "CITY MARKET", "FRESKO LA COMER"]:
            return "SORIANA"
        elif chain in ["CHEDRAUI", "CHEDRAUI SELECTO", "SUPER CHEDRAUI"]:
            return "CHEDRAUI"
        elif chain in ["SUPERMERCADOS LEY", "CASA LEY", "LEY (AUTOSERVICIO)", "LEY EXPRESS"]:
            return "LEY"
        else:
            return chain

    df_filtered["grupo"] = df_filtered["cadenacomercial"].apply(categorize_chain)
    return df_filtered

In [18]:
examinar(pacic)

Productos únicos en la tabla: 
['ACEITE' 'ARROZ' 'ATUN' 'AZUCAR' 'CARNE CERDO' 'CARNE POLLO' 'CARNE RES'
 'CEBOLLA' 'CHILE FRESCO' 'FRIJOL' 'JABON DE TOCADOR' 'JITOMATE'
 'LECHE ULTRAPASTEURIZADA' 'LIMON' 'MANZANA' 'PAN DE CAJA'
 'PAPEL HIGIENICO' 'PASTA PARA SOPA' 'PLATANO' 'SARDINA'
 'TORTILLA DE MAIZ' 'ZANAHORIA' 'HUEVO' 'PAPA' 'LECHE PASTEURIZADA'
 'ESPINACAS' 'JAMON' 'LECHUGA' 'LENTEJA' 'MOJARRA' 'PURE DE TOMATE'
 'SERVILLETAS DE PAPEL BLANCO' 'TOALLA FEMENINA' 'PINA' 'TILAPIA']
Número de productos únicos:  35
Número de registros:  502,599


In [19]:
examinar(df)

Productos únicos en la tabla: 
['A.S.COR' 'ACEITE' 'ACEITE DE OLIVA' 'ACEITUNA' 'ACELGA' 'ACICLOVIR'
 'ACIDO FOLICO' 'ACLIMAFEL' 'ACONDICIONADOR Y ENJUAGUE' 'ADALAT OROS'
 'ADEROGYL 15' 'ADIMOD' 'AFUMIX' 'AFUNGIL' 'AGRIFEN' 'AGUA CON GAS'
 'AGUA SIN GAS' 'AGUACATE' 'ALCOHOL' 'ALEVIAN DUO' 'ALGODON'
 'ALIMENTO PREPARADO PARA NINOS' 'ALIMENTOS PREPADOS' 'ALKA-SELTZER'
 'ALTRULINE' 'AMARYL M' 'AMBROXOL DEXTROMETORFANO' 'AMLODIPINO'
 'AMOXICILINA' 'AMOXICLAV. BID' 'AMPICILINA' 'AMPLIRON DUO' 'ANGELIQ A'
 'ANTIFLU-DES' 'ANTIFLU-DES JR.' 'APROVEL' 'ARAVA' 'ARCOXIA' 'ARROZ'
 'ARTRIDOL' 'ASENLIX' 'ASPIRADORAS' 'ASPIRINA' 'ASPIRINA PROTECT'
 'ATACAND' 'ATACAND PLUS' 'ATORVASTATINA' 'ATUN' 'AUGMENTIN 12 H.'
 'AUTOPISTAS' 'AVAMYS 2A' 'AVELOX 7D' 'AVENA' 'AVODART' 'AZITROMICINA'
 'AZUCAR' 'BACTRIM F' 'BARRA DE SONIDO' 'BATIDORAS' 'BEBIDAS HIDRATANTES'
 'BEDOYECTA' 'BEDOYECTA-TRI' 'BENZONATATO' 'BETABEL'
 'BETAMETASONA/ CLOTRIMAZOL/ GENTAMICINA' 'BEXIDENT ENCIAS' 'BEZAFIBRATO'
 'BIOMETRIX' 'BIOMICS

In [20]:
pacic2=filtro_pacic(df)
print("Número de registros:", format(len(pacic2), ",d"))
print("Productos considerados en tabla pacic que no se encuentran en la pacic2: ")
set(pacic["producto"].unique()) - set(pacic2["producto"].unique())

Número de registros: 1,753,841
Productos considerados en tabla pacic que no se encuentran en la pacic2: 


{'ESPINACAS',
 'JAMON',
 'LECHE PASTEURIZADA',
 'LECHUGA',
 'LENTEJA',
 'MOJARRA',
 'PINA',
 'PLATANO',
 'PURE DE TOMATE',
 'SERVILLETAS DE PAPEL BLANCO',
 'TILAPIA',
 'TOALLA FEMENINA'}

In [21]:
#Filtro de supermercados
pacic2=filtro_supermercado(pacic2)
examinar(pacic2)

C:\Users\claudio.pacheco\AppData\Local\Temp\ipykernel_27940\2895199600.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["grupo"] = df_filtered["cadenacomercial"].apply(categorize_chain)


Productos únicos en la tabla: 
['ACEITE' 'ARROZ' 'ATUN' 'AZUCAR' 'CARNE CERDO' 'CARNE POLLO' 'CARNE RES'
 'CEBOLLA' 'CHILE FRESCO' 'FRIJOL' 'HUEVO' 'JABON DE TOCADOR' 'JITOMATE'
 'LECHE ULTRAPASTEURIZADA' 'LIMON' 'MANZANA' 'NARANJA' 'PAN DE CAJA'
 'PAPEL HIGIENICO' 'PASTA PARA SOPA' 'SARDINA' 'TORTILLA DE MAIZ'
 'ZANAHORIA' 'PAPA']
Número de productos únicos:  24
Número de registros:  1,485,321


In [22]:
#Salvar el dataframe en un archivo csv
pacic2.to_csv("pacic.csv", index=False)

In [23]:
#Obtener precio promedio de la canasta de 24 productos por grupo con formato
average_price_by_group = pacic2.groupby("grupo")["precio"].mean()
average_spend_by_group = average_price_by_group * 24

formatted_spend = average_spend_by_group.map("${:,.2f}".format)
for group, spend in formatted_spend.items():
    print(f"{group} = {spend}")

CHEDRAUI = $1,207.14
LEY = $1,177.09
SORIANA = $1,161.73
WALMART = $1,263.50


In [24]:
#Datos a cuadro de resultados
df_spend = formatted_spend.reset_index()
df_spend.columns = ["grupo", "costo"]

# Tabla
fig = go.Figure(data=[go.Table(header=dict(values=["<b>Grupo</b>",
                        "<b>Costo promedio</b>"
                          ],
                fill_color='#9d2449',
                align='center',
                font_color='white',
                font_family="Montserrat",
                font_size=20),
    cells=dict(values=[df_spend["grupo"], df_spend["costo"]],
fill_color='white',
               align='center',
           font_family="Montserrat",
           font_size=18,
           height=30,
           line_color='black',
           
           ))
])

#Título
fig.update_layout(
    showlegend=False,
    title_text="<b>Costo promedio de la canasta PACIC por grupo, enero-septiembre 2023<br>",
    title_font_family="Montserrat",
    title_font_size=25
)


#Fuente
fig.add_annotation(text="Secretaría de Economía. Dirección General de Competitividad y Competencia con información de PROFECO. Quién es quién en los precios. Datos abiertos.",
                  x=0, y=-0.09, showarrow=False,
                   font_family="Century Gothic")

#Salvar la tabla como PNG 
pio.write_image(fig, "profeco1/costocanasta_grupo.png", width=1000, height=450,scale=1)